In [1]:
%matplotlib notebook
from ABME import *
from agents import *

In [2]:
def simulate(agents, tracker, gridsize=50, iterations=1000):
    #Setup objects
    # First create a grid for the model and the model itself
    g = Grid(50);
    m = Model();
    m.dead = []; #Create a list for the dead
    #and add the grid to the model
    m.set_grid(g);

    # Create a visualizer and a tracker
    vis = Visualizer(m);
    tracker.model = m;

    # And track population using the default tracker
#     tracker.add_tracker('Population', lambda m: len(m.agents)); # lambda model: len(model.agents)

    #Same as: def foo(model): return len(model.agents);
    #         tracker.add_tracker('Population', foo);

    #Later, tracker['Population'] can be used to get the population data

    #Create 2 resources, these are automatically added to the grid [g]
    r1 = Resource("Kerbonite", g);
    r2 = Resource("Ebolite", g);

    # Create a capacity function and regrowth function
    cap=lambda x:np.round(4*np.exp(-(x**2)/(2*(0.08)**2))); #  x=distance from center(given in grow)
    reg=lambda x:0.5; # Uniform growth of 0.5, x is ignored

    # Set capacity
    #    scale = true makes the distance from center from [0, 1]
    #    Making a bigger map thus keeps the ratios of resources equal
    #    scale = True is default
    r1.set_capacity(20,15, fun=cap, scale=True);
    r1.set_capacity(35,35, fun=cap, scale=True);
    r2.set_capacity(35,15, fun=cap, scale=True);
    r2.set_capacity(20,35, fun=cap, scale=True);

    # grow to the given capacity (such that there are resources available in the first tick)
    r1.grow(20,20, fun=cap);
    r2.grow(35,20, fun=cap);
    r1.grow(35,35, fun=cap);
    r2.grow(20,35, fun=cap);


    #add agents
    for agent in agents:
        m.add_agent(agent);

    # or add a single test agent
    # a = SugerAgent();
    # #Modify random variables before adding the agent
    # # Modifying other variables which are randomly initialized, like vision, have no effect
    # # before the add_agent call has been made as this call initializes these variables
    # a.wealth_low = 10;
    # a.wealth_high = 11;
    # #Add the agent
    # #    This call also initializes the agents values based on the _low _high
    # #    From nowon, modifying things like wealth_low, has no effect
    # m.add_agent(a, position=[35,35]); 
    # #Set variables of this agent
    # a.max_age = 20000;
    # a.metabolism = 0.1;
    # a.vision = 30;

    # simulate
    for i in range(iterations):
        # Core loop
        r1.grow(20,15,fun=cap);
        r2.grow(35,15, fun=cap);
        r1.grow(35,35,fun=cap);
        r2.grow(20,35, fun=cap);
        # following 3 calls is the same as m.step()
        m.step_move();
        m.step_act();
        m.track(); #Using all trackers, save data

        #Break if all agents have dieded
        if len(m.agents) == 0:
            break;
            
    return m;

In [3]:
def recombinate(g1, g2, mutation_std=0.1):
    #Take average
    ng = (np.array(g1) + np.array(g2)) / 2;
    #add mutation
    for i in range(len(ng)):
        ng[i] = np.random.normal(ng[i], mutation_std);
    
    return ng;

def create_new_gen(population, size, privilaged=10, self_replication=5, num_mates = 10, offspring_per_mate=3, fill=True, mutation_rate=0.1):
    new_pop = [];
        
    #Sort current population on fitness
    population.sort(key=lambda a: a.fitness(), reverse=True);

    #Number of privilaged members of population, the progenitors of the next generation
    # They will now mate with num_mates
    if privilaged > len(population):
        privilaged = len(population);
    for i in range(privilaged):
        progenitor = population[i];
        for j in range(num_mates):
            mate = np.random.choice(population);
            for k in range(offspring_per_mate):
                new_genome = recombinate(progenitor.genome, mate.genome, mutation_rate);
                child = EvolutionAgent();
                child.genome = new_genome;
                new_pop.append(child);
                
    #Self replication
    for i in range(privilaged):
        a = population[i];
        for j in range(self_replication):
            new_genome = recombinate(a.genome, a.genome, mutation_rate);
            child = EvolutionAgent();
            child.genome = new_genome;
            new_pop.append(child);
    
    #Add random childs till population size is equal again
    for i in range(len(new_pop), size):
        new_pop.append(EvolutionAgent());

    return new_pop;

Create P1, the initial population of agents

In [4]:
# Settings
N = 1000; #Population size
wealth_low = 10;
wealth_high = 15;

In [5]:
#Create population, note that you cannot call fitness before they are in the simulation
population = [];
for i in range(N):
    a = EvolutionAgent();
    a.wealth_low = wealth_low;
    a.wealth_high = wealth_high;
    population.append(a);

Now setup a simulation and keep simulating and creating new offsprings

In [ ]:
sim_tracker = Tracker(); #logs data per sim iteration
evo_tracker = Tracker(); #logs data per evolution

evo_tracker.add_tracker("Fitness", lambda m: [a.fitness() for a in m.agents]);

In [ ]:
for generation in range(50):
    display.clear_output(wait=True);
    model = simulate(population, sim_tracker, gridsize=50, iterations=1000);

    #create new population
    cpop = [];
    for a in model.agents:
        cpop.append(a);
        print(a)
    for a in model.dead:
        cpop.append(a);
    model.agents = cpop;
    
    print("Generation %i" % generation);
    print("Mean fitness: %f (%f)" % (sum([a.fitness() for a in cpop]), np.std([a.fitness() for a in cpop])));
    print("Survived %i iterations" % model.tick)

    population = create_new_gen(cpop, N, \
                                mutation_rate=0.1, \
                                privilaged=100, \
                                offspring_per_mate=2, \
                                num_mates=2, \
                               self_replication=5);
    
#SHow final generations state
vis = Visualizer(model);
vis.plot_grid();
vis.plot_wealth_hist();
vis.plot_needs_metabolism();
vis.show();

Agent 181
	Metabolism: 1.667294
		need 0.020909% Ebolite
		need 99.979091% Kerbonite
	Age: 1000
	Vision (1.334589): 1
	wealth (1.000000-3.000000):
		Ebolite: 0.651389
		Kerbonite: 2332.054330
	Genome:
		vision: 1.334589
		alpha: 1.719924
		beta: -0.166696
		gamma: 0.090189
		needs1: 0.000209
		needs2: 0.999791

Agent 420
	Metabolism: 1.520286
		need 0.027890% Ebolite
		need 99.972110% Kerbonite
	Age: 1000
	Vision (1.040572): 1
	wealth (1.000000-3.000000):
		Ebolite: 0.575989
		Kerbonite: 2474.138069
	Genome:
		vision: 1.040572
		alpha: 0.511577
		beta: -0.200911
		gamma: 0.052229
		needs1: 0.000279
		needs2: 0.999721

Agent 531
	Metabolism: 1.726694
		need 30.116672% Ebolite
		need 69.883328% Kerbonite
	Age: 1000
	Vision (1.453388): 1
	wealth (1.000000-3.000000):
		Ebolite: 481.977251
		Kerbonite: 795.328819
	Genome:
		vision: 1.453388
		alpha: 1.173332
		beta: -0.892738
		gamma: -0.056544
		needs1: 0.301167
		needs2: 0.698833

Agent 898
	Metabolism: 1.444201
		need 51.300610% Ebolite
